In [1]:
import numpy as np
import xsimlab as xs
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Let's take notes again,

so I should put every process into a process class

but that creates problems! what about parameters?
- i need to create an instance of the process class, within the components, actually... (?) 

also, what about the fluxes?



This is a central question right now, what is the best way to structure the package, based on xsimlab

xsimlab was not written for my kind of applications, it actually is more suitable for agent-based models
i'm doing equation based modeling..

### and at some point in my model I need to define the *_fluxes_*

but where is best? prob in the environment, right? but then what about modification of the ODE?

## what if I put the ODE in a seperate class? would that make any sense?

MODELICA - interesting source on this: https://portal.research.lu.se/ws/files/4779081/8571901.pdf

It defines all components + params + additionally the model equation! (this is missing from xsimlab, somehow--)


In [2]:
IDEa: can keep equation construction separate from model assembly, check all components present before?

Object `before` not found.


In [3]:
could also keep all "MATHS FUNX" outside of xsimlab..

would that increase usability??

I need some way to contruct ODE systems, one component has one flux, right?

so perhaps that is the way, have each component assemble it's flux, and that is passed on to final model

but then how do components interact? components inherits all processes that are part of it's flux, and then later find duplicities automatically?

Nutrient . flux['grazing'] - flux['Z_mortality']

SyntaxError: invalid syntax (<ipython-input-3-7deae310912d>, line 1)

In [ ]:
@xs.process
class Phytoplankton:
    Uptake = xs.foreign(PhytoStdLib,'MichaelisMenten')
    
    

In [ ]:
- write bootstrap model

- then write basic model

and only then adapt all other models

# keep in mind that this is how end user would use models:

In [ ]:
import xsimlab as xs

from ..processes.boundary import BorderBoundary
from ..processes.channel import (StreamPowerChannel,
                                 DifferentialStreamPowerChannelTD)
from ..processes.context import FastscapelibContext
from ..processes.flow import DrainageArea, SingleFlowRouter, MultipleFlowRouter
from ..processes.erosion import TotalErosion
from ..processes.grid import RasterGrid2D
from ..processes.hillslope import LinearDiffusion, DifferentialLinearDiffusion
from ..processes.initial import (BareRockSurface,
                                 Escarpment,
                                 FlatSurface,
                                 NoErosionHistory)
from ..processes.main import (Bedrock,
                              StratigraphicHorizons,
                              SurfaceTopography,
                              SurfaceToErode,
                              TerrainDerivatives,
                              TotalVerticalMotion,
                              UniformSedimentLayer)
from ..processes.marine import MarineSedimentTransport, Sea
from ..processes.tectonics import (BlockUplift,
                                   SurfaceAfterTectonics,
                                   TectonicForcing,
                                   TwoBlocksUplift)


# ``bootstrap_model`` has the minimal set of processes required to
# simulate on a 2D uniform grid the evolution of topographic surface
# under the action of tectonic and erosion processes. None of such
# processes are included. It only provides the "skeleton" of a
# landscape evolution model and might be used as a basis to create
# custom models.

bootstrap_model = xs.Model({
    'grid': RasterGrid2D,
    'fs_context': FastscapelibContext,
    'boundary': BorderBoundary,
    'tectonics': TectonicForcing,
    'surf2erode': SurfaceToErode,
    'erosion': TotalErosion,
    'vmotion': TotalVerticalMotion,
    'topography': SurfaceTopography,
})

# ``basic_model`` is a "standard" landscape evolution model that
# includes block uplift, (bedrock) channel erosion using the stream
# power law and hillslope erosion/deposition using linear
# diffusion. Initial topography is a flat surface with random
# perturbations. Flow is routed on the topographic surface using a D8,
# single flow direction algorithm. All erosion processes are computed
# on a topographic surface that is first updated by tectonic forcing
# processes.

basic_model = bootstrap_model.update_processes({
    'uplift': BlockUplift,
    'surf2erode': SurfaceAfterTectonics,
    'flow': SingleFlowRouter,
    'drainage': DrainageArea,
    'spl': StreamPowerChannel,
    'diffusion': LinearDiffusion,
    'terrain': TerrainDerivatives,
    'init_topography': FlatSurface,
    'init_erosion': NoErosionHistory
})

In [ ]:
# NOTE TO SELF:
Encapsulation as used by xsimlab does not work with equations!

"""Due to the acausal nature of equations it is impossible to enforce encapsulation of
equations in partial components for a library developer who provides partial models. - Modelica Thesis"""

In [ ]:
SO: somehow, equations need to be generated from higher_level_process!
    
    
    hm, instead of having higher level process, how about all components that modify ODE have a mixin class, that keeps track of all the ODE components, and then later calling that
    parent class functionality returns all components fluxes and assembles ODE
    
    how to specify interaction? this needs to be set down in this class, and then specified in each process, like [1,'p',-1,'N']
    to make this extensible there needs to be a conversion step between each process (the possibility should be there)
    
    need to propagate component properties & parameters at Model instance level to all components, similar to how fs_context[''] returns all relevant properties of the fastscape model